In [74]:























from mistune.plugins.abbr import process_text
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import smtplib

In [75]:
url = 'https://www.amazon.in/Redmi-A4-5G-Sparkle-Charging/dp/B0DLW427YG/ref=sr_1_1_sspa?_encoding=UTF8&content-id=amzn1.sym.bd428427-e2fa-455b-845a-a54edb5f536f&dib=eyJ2IjoiMSJ9.Zu16M0f4gcJTITvmFzGqs09EXESHBn36UX9vRwLtp43PhS-3_IY2Fs73Ku6LbRlMIJ-BGLDG7ZZd2ctqJbqxv7vk6mrTF2l87iQWhN2qtRYWXCRutAfVXjCa6s9k8lYTdcgifhH_faDWuAyM1EmFgkLcz5HCH5PoTuKiJkZjPjGH1xy1OtH05DxdSO3P-dJ1heNXxYWzobMdKRP3M5E4Ta0jb0TVF9ZnoKSzGwgWY6u7GVXjITUOJHg7UoPtX3GxEtJrG6wAvgL4S38ajygckC1WHO4hX_D2y3HIHFLe8Lc.L5Y8AffFc-O4ioP4KMYOvwBicFzKvGvZrsYAoA_GYXU&dib_tag=se&pd_rd_r=56178e86-1b26-4ccb-a9fe-28a25be7649f&pd_rd_w=A11pD&pd_rd_wg=MYv29&qid=1746590515&refinements=p_36%3A-1030000&rnid=1318502031&s=electronics&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl&th=1'

In [76]:
TARGET_PRICE = 10000

In [77]:
CHECK_INTERVAL = 300

In [78]:
def initialize_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Run in headless mode (no browser window)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.100.0")

    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [79]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_product_details():
    driver = None
    try:
        driver = initialize_driver()
        driver.get(url)

        wait = WebDriverWait(driver, 15)

        # Extract title
        title_element = wait.until(EC.presence_of_element_located((By.ID, "productTitle")))
        title = title_element.text.strip()
        print("Title:", title)

        # Wait for the price container and extract visible text
        price_container = wait.until(EC.visibility_of_element_located((
            By.CLASS_NAME, "priceToPay"
        )))
        price_text = price_container.text.strip()

        # Example result: "₹7,999"
        if price_text:
            # Extract digits from ₹7,999 safely
            import re
            price_digits = re.sub(r'[^\d]', '', price_text)
            if price_digits:
                price = int(price_digits)
                print("Price:", price)
                return title, price

        print("Price element found but text is empty")
        return title, None

    except Exception as e:
        print(f"Error: {e}")
        return None, None

    finally:
        if driver:
            driver.quit()


In [80]:
  get_product_details()

Title: Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging
Price: 7999


('Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging',
 7999)

In [81]:
def amazon_send_email(title, price):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()

    server.login('divyansh.raghav@contify.com', 'bimymxkesiikjcey')

    subject = f"Price Alert: {title} is now Rs.{price}"
    body = f"The price of {title} has dropped to Rs.{price}!\n"

    message = f"Subject: {subject}\n\n{body}"

    server.sendmail(
        'divyansh.raghav@contify.com',
        'raghavdivyansh9@gmail.com',
        message
    )

    server.quit()
    print(f"Email sent at {datetime.datetime.now()}")

In [82]:
def check_price_once():
    print(f"Checking price at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    title, price = get_product_details()

    if title and price:
        print(f"Product: {title}")
        print(f"Current Price: Rs.{price}")
    else:
        print("Could not retrieve product details")

    return title, price

In [83]:
title, price = check_price_once()

Checking price at 2025-05-08 12:58:08
Title: Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging
Price: 7999
Product: Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging
Current Price: Rs.7999


In [84]:
amazon_send_email(title, price)

Email sent at 2025-05-08 12:58:24.244959


In [85]:
get_product_details()

Title: Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging
Price: 7999


('Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging',
 7999)